## Cluster-Based Summarization 分群摘要方法
Cluster-based summarization的主要概念是使用相似計算(similarity measures)把一個檔案的句子分成不同群(clusters)依照句子的相似度。因此，不同的群會代表不同的話題，然後從每群抽出得到高分的句子就能夠做成摘要。在這個示範裏，我們用的分群方法是k-means clustering，相似計算方法為cosine similarity。K-means clustering會把文檔的句子分成k群，然後每個群會各計算出一個群中心(centroid)。利用cosine similarity的相思度計算，越類似群中心的句子會得到越高的分數。最後，每群得分最高的句子會被選為做成有k句子的摘要。

Cluster-based的摘要方法的好處是可以代表多樣的句子並減少冗餘。除了單文件摘要以外，分群摘要方法也可以用來做多文件長文摘要。

## 讀取資料
- 讀取爬文和下載好的100篇文章

In [1]:
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [2]:
# 讀取爬下來的文集
import pandas as pd
news100 = pd.read_excel('appledaily_politinews100.xlsx')
news100.head()

,category,content,title
0,政治,總統蔡英文卸任民進黨主席，昨在臉書po給黨員的一封信，信中說，民進黨會展開一場轟轟烈烈的檢討...,蔡英文要轟轟烈烈檢討民進黨 林濁水酸：民主國家沒聽過
1,政治,民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變...,他列民進黨10大豬隊友 「不下架將賠上2020」
2,政治,2018年地方選舉國民黨拿下空前勝利，執政地區從6縣市竄升為15縣市，昨中常會上國民黨提出選...,選戰檢討報告出爐 國民黨：落選者「缺乏對外拓票企圖心」
3,政治,準高雄市長韓國瑜的愛女韓冰昨(28日)受邀上政論節目《54新觀點》，首度與民進黨台北市議員王...,不用跳愛河！韓冰邀替高雄拚觀光 王世堅靦腆：可以廢物利用
4,政治,點我直接觀看 被操弄的民調？《蘋果》數據解密,【民調解密3】林佳龍敗在紫爆 盧秀燕跌破眼鏡大勝


## 分詞/分句子，把文章轉換成Vector
- 清理文章，再分成句子
- 把句子轉換成count vector

In [3]:
# 選擇文章，把文章分成句子
docNum = 31
testDoc = news100['content'][docNum]
testTitle = news100['title'][docNum]

import re
def splitSentences(document):
    #for sent in re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U):
    for sent in re.findall(u'[^!?。\!\?]+[!?。\!\?]?', document, flags=re.U):
        yield sent

import jieba
sentenceDoc = []
for sent in list(splitSentences(testDoc))[:-1]:
    sentenceDoc.append(' '.join(jieba.cut(sent)))

# 原本文章
testDoc

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YB0002~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.754 seconds.
Prefix dict has been built succesfully.


'今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。行政院發言人Kolas Yotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。NCC及中華電信暫未對此做進一步說明。陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。（林惟崧、鄭鴻達／台北報導）出版：1454更新：1942（新增行政院說法） \xa0想知道更多，一定要看……【獨家】投票日遭境外駭客攻擊\u3000中選會網站險癱瘓看了這則新聞的人，也看了……老里長險勝1票\u3000美女里長候選人要求驗票發祭品囉！\u3000韓總雞排逾千

In [4]:
# 分詞後的文章
sentenceDoc

['今年 九 合一 大選 投票 狀況 多 ， 據 了解 ， 中選會 官方 網站 在 投票 日當天 曾 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊 ， 所幸 在 負責 網路 維護 中華電信 應變 下 ， 未進 一步 造成 影響 。',
 '中選會 副 主委 陳 朝 建今 證實 曾 遭駭客 攻擊 ， 不過 計票 網路 是 封閉 系統 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 因此 沒有 任何 影響 ， 境外 網路 干擾 也 並未 影響 投票 進程 。',
 '行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下 來 將針 對 有形 的 選務如 投票 排隊 問題 ， 以及 無形 的 選務 如資 安防 護進行 檢討 。',
 '不過 ， 投票 日當天 開票 進程 緩慢 ， 台北市 選情 更 創下史 上 第一次 開票 長 達 9 小時 的 紀錄 ， 陳 朝建 說明 ， 開票 時間 長 是 因為 投票 時間 延 後 ， 計票 不受 駭客 影響 ， 開票 也 自然 不受 此 影響 。',
 '據 內部 消息 指出 ， 投票 日當天 曾 出現 從 境外 進行 分散式 阻斷 服務 （ DDoS ） 攻擊 ， 企圖 癱瘓 中選會 網站 ， 不過 在 中華電信 、 中選會 緊急 處理 後 ， 阻斷 來 自國外 異常 流量 ， 在 短 時間 獲得 解決 ， 一般 民眾 並沒有 發現 特殊 異樣 。',
 'NCC 及 中華電信 暫未 對 此 做 進 一步 說明 。',
 '陳 朝建 表示 ， 受 攻擊 的 是 中 選會 官方 網站 ， 當日 開票 的 計票 網路 是 獨立 的 封閉 網路 ， 並 非公 開網 路 ， 無法 由 外界 接取 ， 完全 未有 任何 影響 ； 中華電信 團隊 也 是 24 小時 監控 與 防護 作業 ， 也 沒 有 受到 任何 影響 。',
 '陳 朝建 說 ， 受到 攻擊 的 官網 ， 當日 皆 有 隨時 防衛 ， 中華電信 團隊 24 小時 監控 與 防護 作業 ； 故當 日 遭到 來 自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊時 ， 在 負責 網路 維護 中華電信 的 緊急 應變 下 ， 並

In [5]:
# 把文章轉換成 count vector
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
doc_vector = vect.fit_transform(sentenceDoc)

In [6]:
# 計算 vector 之間的相似度(cosine similarity)
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(doc_vector)
cs

array([[1.        , 0.29032258, 0.13116517, 0.18146655, 0.37387825,
        0.16064387, 0.23308662, 0.44901326, 0.18247917, 0.17255919],
       [0.29032258, 1.        , 0.13116517, 0.23331413, 0.17255919,
        0.        , 0.52444489, 0.1796053 , 0.27371875, 0.0862796 ],
       [0.13116517, 0.13116517, 1.        , 0.05270463, 0.11694107,
        0.        , 0.08885233, 0.06085806, 0.34780417, 0.46776428],
       [0.18146655, 0.23331413, 0.05270463, 1.        , 0.13867505,
        0.06454972, 0.2575611 , 0.09622504, 0.        , 0.04622502],
       [0.37387825, 0.17255919, 0.11694107, 0.13867505, 1.        ,
        0.07161149, 0.07792865, 0.26688026, 0.18302666, 0.17948718],
       [0.16064387, 0.        , 0.        , 0.06454972, 0.07161149,
        1.        , 0.07254763, 0.1490712 , 0.        , 0.        ],
       [0.23308662, 0.52444489, 0.08885233, 0.2575611 , 0.07792865,
        0.07254763, 1.        , 0.45962736, 0.22662355, 0.05195243],
       [0.44901326, 0.1796053 , 0.0608580

In [7]:
# 選擇摘要的長度
sentCount = 3

# 用 k-means clustering 把每個 vector 分成三群
from sklearn.cluster import KMeans
km = KMeans(n_clusters=sentCount, init='k-means++')
c = km.fit_predict(cs)
c

array([0, 2, 1, 2, 0, 0, 2, 0, 1, 1])

In [8]:
import numpy as np
np_doc = np.array(sentenceDoc)
print(np_doc[c==1])

['行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下 來 將針 對 有形 的 選務如 投票 排隊 問題 ， 以及 無形 的 選務 如資 安防 護進行 檢討 。'
 '另外 ， Kolas 表示 ， 中選會 的 官網 在 選舉 當天 上午 6 時 56 分至 7 時 01 分期 間 ， 遭受 大規模 境外 DDoS 攻擊 ， 而 中 選會 的 系統 是 封閉 系統 ， 防護 非常 嚴密 ， 因此 沒 被 攻擊 ， 而當 因中 選會 在 選前 即 做好 縝密 的 防護 ， 當天 其實 沒因 網路 攻擊發 生意 外狀況 ， 中選會 的 資安 管理 有 成功 防堵 攻擊 。'
 'Kolas 表示 ， 所幸 這次 在 事前 有 預見 可能 的 危險 ， 中選會 的 資料 庫 、 網站 等 都 沒 遭受 損害 ， 而 在 資訊 時代 ， 行政院 希望 在 這波 中選會 的 選務 檢討 中 ， 不 只針 對 有形 被 詬 病 的 排隊 投票 問題 ， 無形 的 選務 如資 安防 護 ， 也 會 進行 檢討 、 加強 。']


## 分群
- 把每個句子分成群依照句子跟中心的相似度

In [9]:
# 把每個分群依照跟中心的相似度排列從大到小，找出最適合做摘要的句子
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
order_centroids

array([[0, 7, 4, 5, 6, 1, 8, 3, 9, 2],
       [2, 9, 8, 1, 0, 4, 6, 7, 3, 5],
       [6, 1, 3, 7, 0, 8, 4, 2, 9, 5]], dtype=int64)

In [10]:
# 找出每個群的前三個句子(i.e. 最重要/相似的句子)
centroid_top3 = km.cluster_centers_.argsort()[:, :6:-1]
centroid_top3

array([[0, 7, 4],
       [2, 9, 8],
       [6, 1, 3]], dtype=int64)

In [11]:
# 依照每個群print句子
print('標題: ' + '《' + testTitle + '》')
clusterIndex = 0
for cluster in centroid_top3:
    print('Cluster: ' + str(clusterIndex))
    for sent in cluster:
        print('Sentence ' + str(sent), sentenceDoc[sent].replace(' ','').strip())
    clusterIndex += 1
    print('')

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
Cluster: 0
Sentence 0 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 7 陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。
Sentence 4 據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。

Cluster: 1
Sentence 2 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
Sentence 9 Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。
Sentence 8 另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。

Cluster: 2
Sentence 6 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。
Sentence 1 中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。
Sentence 3 不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影

## 長文摘要

In [12]:
# 找出每個cluster最像中心的三個句子做摘要
summarySentences = []
for cluster in order_centroids:
    summarySentences.append(cluster[0])

print('標題: ' + '《' + testTitle + '》')
for sentence in sorted(summarySentences):
    print('Sentence ' + str(sentence) + '\n', sentenceDoc[sentence].replace(' ','').strip())

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
Sentence 0
 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 2
 行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
Sentence 6
 陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。


**會提高摘要效果的改進**
1. 更好的分詞/分句子會有更好的效果和摘要
2. 更進階的方法計算句子的重要性(這個示範只有用詞頻計算相似度)
3. 需要選擇恰當的cluster size(太多clusters摘要會包括多餘的句子，太少clusters會忽略一些重要的juzi)
    - 不同的文章會需要不同的cluster size

**其他摘要例子**

In [13]:
# cluster-based 摘要，輸入文章的號碼和摘要長度
def clusterSummarizer(docNumber, sumLength):
    testDoc = news100['content'][docNumber]
    testTitle = news100['title'][docNumber]

    sentenceDoc = []
    for sent in list(splitSentences(testDoc))[:-1]:
        sentenceDoc.append(' '.join(jieba.cut(sent)))
    
    doc_vector = vect.fit_transform(sentenceDoc)
    cs = cosine_similarity(doc_vector)
    
    km = KMeans(n_clusters=sumLength, init='k-means++')
    km.fit_predict(cs)
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    
    summarySentences = []
    for cluster in order_centroids:
        summarySentences.append(cluster[0])
        
    print('標題: ' + '《' + testTitle + '》')
    print(testDoc, '\n')

    print('標題: ' + '《' + testTitle + '》')
    for sentence in sorted(summarySentences):
        print('Sentence ' + str(sentence) + '\n', sentenceDoc[sentence].replace(' ','').strip())

In [14]:
# 稍微短一點的文章，第99篇
clusterSummarizer(99,3)

標題: 《柯、侯都愛！副市長虛位以待　邱豐光人氣破表》
選戰落幕，各縣市首長將於12月25日就職，一場搶人大戰正式展開！台北市警察局前局長邱豐光，去年9月被調往警政署任副署長，外界多以「明升暗降」看待。如今選舉結束，台北市長柯文哲、新北市長當選人侯友宜不約而同看上邱，冀望找他來當副市長。《周刊王》報導，柯文哲原本不喜歡邱豐光，後來發現邱「很好用」，不僅對議會的溝通效果比副市長強，邱在商界人脈廣，只要柯P開口，邱使命必達。侯友宜雖是邱豐光警大學弟，但據報導，早在侯決定參選時，已私下探詢邱任副手的意願，當時邱力挺，「只要侯開口，絕對會去。」對此傳聞，柯、侯幕僚均表示目前沒討論。邱豐光回覆「空穴來風」，目前最重要的是做好警政工作。（王嘉慶／綜合報導）想知道更多，一定要看……【君子片】密約曝光！韓國瑜、陳其邁　選前3度交換協議 

標題: 《柯、侯都愛！副市長虛位以待　邱豐光人氣破表》
Sentence 1
 如今選舉結束，台北市長柯文哲、新北市長當選人侯友宜不約而同看上邱，冀望找他來當副市長。
Sentence 3
 侯友宜雖是邱豐光警大學弟，但據報導，早在侯決定參選時，已私下探詢邱任副手的意願，當時邱力挺，「只要侯開口，絕對會去。
Sentence 4
 」對此傳聞，柯、侯幕僚均表示目前沒討論。


In [15]:
# 稍微長一點的文章，第40篇
clusterSummarizer(40,3)

標題: 《坦言「​總統大選光環已過」　蔡英文座談會聚焦基層》
總統蔡英文下午前往宜蘭展開「傾聽之旅」第二站，歷經2小時的座談會，下午5時許蔡英文等人離開會場。被問到是否會參選2020、非核家園2025等問題蔡英文笑而不答，而被問到農委會主委林聰賢是否應該下台一事，她一度欲言又止，最後還是上車離去。 與會的宜蘭縣小英之友會理事曹光炎說，總統放下身段傾聽基層聲音，他相信基層聲音總統都聽到了，民進黨也會虛心改進，尤其是照顧弱勢的農漁工，還說「民進黨的失敗，不是因為國民黨的強，而是因為沒有傾聽人民的聲音」。會中並沒有針對特定人士是否下台進行討論。 民進黨宜蘭縣黨部主委黃進財也說，會中探討執政上遇到、需要改進的問題，並沒有提到行政官員是否下台的部分。他說，各領域都有探討，包括一例一休、農漁會與空污法問題，都有討論到，而執政上沒有立即接地氣與民意站在一起都有做深切檢討。 黃進財也表示，座談會中也多所提及農業政策，包括農地、農舍等議題及農漁民收入、農業產銷穩定等問題都有熱烈討論，總統也相當重視。 另外，宜蘭市長江聰淵表示，在於內閣部分有特別提到，人才尋覓是相當重要課題，過去都被批評是用老宅男、老藍男等問題，人才尋找是當務之急，對於人才培養、部會的養成，因此才會繼續慰留陳菊及賴清德繼續承擔責任。 對於局部調整，也聽很多基層鄉親意見，包括勞工、漁民、政策等問題，中央部會未在第一時間解決，總統承諾未來會經常下鄉聽取民眾聲音，會透過她直接與部會溝通，而非侷限在建議案到她手中後才與部會商量，將來會以下鄉的方式，解決民眾需求，直接了解民眾心聲。 另外，江聰淵也說，許多政策議題上很多有爭議性，總統也處於兩難，包括廢死刑屬國際議題，國際友邦要求不能執行死刑，但國人對於廢死也有反對立場，公投案也處於對立，造成執政者負擔。如今執政運作上問題浮現，須面對問題並解決。 江聰淵說，蔡英文在會中表示「總統大選時的光環已經過去了」，會虛心檢討，鼓勵基層民眾，重拾信心，改革與政策推動，會與地方謀定而後動，最重要的民生議題，包括一例一休等相關政策，會再找出明確性的推動方向。 對於縣黨部輔選不力部分，確實有人提議檢討，江聰淵表示，會中也曾討論到農委會主委，總統說前後兩任均是出身農業大縣的縣長，她給予充分授權，未來要再尋覓有敏感性、能替政府、百姓解決問題的人才，是總統擔憂的部分，但她相信有此時間歷練，各部會都進入狀況

In [16]:
# 效果不太好的摘要，第1篇
clusterSummarizer(1,3)

標題: 《他列民進黨10大豬隊友　「不下架將賠上2020」》
民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變的人就是她自己，「責任由我一肩扛起，同志之間不必互相為難。」前立委孫大千則說，「蔡總統又不能辭職下台，那要如何一肩扛起所有責任？這不又是一句空話嗎？」孫大千今天上午在臉書發文，倘若民進黨真的希望能夠救亡圖存，力挽狂瀾，建議在最短的時間內，不論是用勸退、勸辭、或是撤換的方式，讓下列十大「豬隊友」下架，「如果捨不得這十個人，保證會賠上民進黨2020年的政權。」第一，副總統陳建仁理由：823水災時，當南部鄉親還泡在水裡，但是這個人卻在金門玩好玩滿了3天。第二，總統府秘書長陳菊理由：把高雄市整到「又老又窮」，和滿朝盡是「菊家班」。第三，行政院長賴清德理由：「幹話王」的形象早已深植人心。第四，行政院發言人谷辣斯理由：不但有酒駕前科，更不忘記把政府提供給弱勢原住民的法扶資源「用好用完」。第五，駐日代表謝長廷理由：有空回台輔選黨內同志，卻無心援助台灣旅客。第六，交通部長吳宏謀理由：高雄市五千個「天坑」的推手，以及普悠瑪事件的政治責任。第七，農委會副主委李退之理由：豐年社社長汪文豪的「被請辭事件」。第八，監察委員陳師孟理由：替民進黨把司法檢調系統，幾乎一次羞辱、踐踏、得罪完畢。第九，北農總經理吳音寧理由：看到這位有蔡英文當靠山的「高級實習生」，只會讓年輕人一肚子怨氣。第十，中華電信董事長鄭優理由：從「499之亂」，到後續的內部爭議連環爆，他必然將會成為在野黨的下一個靶子。（即時新聞中心／綜合報導）看了這則新聞的人，也看了……14年前撞死人判緩刑 　韓國瑜下午將出面說明不用跳愛河！韓冰邀替高雄拚觀光　王世堅靦腆：可以廢物利用蔡英文要站第一線　林濁水諷：賴清德變隨身特助 

標題: 《他列民進黨10大豬隊友　「不下架將賠上2020」》
Sentence 1
 」前立委孫大千則說，「蔡總統又不能辭職下台，那要如何一肩扛起所有責任？這不又是一句空話嗎？」孫大千今天上午在臉書發文，倘若民進黨真的希望能夠救亡圖存，力挽狂瀾，建議在最短的時間內，不論是用勸退、勸辭、或是撤換的方式，讓下列十大「豬隊友」下架，「如果捨不得這十個人，保證會賠上民進黨2020年的政權。
Sentence 4
 第三，行政院長賴清德理由：「幹話王」的形象早已深植人心。
